In [1]:
import sage.schemes.elliptic_curves.isogeny_small_degree as isd

ZZR = PolynomialRing(ZZ, name='XX')

def swudataToStr(data,p,extdegree):
    res="{"
    tostr=lambda x,p:str(-(p-x)) if abs(-(p-x))<20000 else (hex(x) if x>20000 else str(x))
    for i in list(data.items()): 
                chars=[["",""],["[","]"]]
                if  i[0]in ('swuA','swuB'):
                    _x=i[1]
                    if len(_x)<extdegree:_x=_x+[0]*(extdegree-len(_x))
                    _=(chars[len(_x)>1][0]+"".join(tostr(j,p)+"," for j in _x))[:-1]+chars[len(_x)>1][1]+","                                     
                else:
                    if i[0]=="Z":
                        _=("["+"".join(tostr(j,p)+"," for j in i[1]))[:-1]+"]," if len(i[1])>1 else str(i[1][0])+","
                    else:
                        if i[0]=="Degree":_=str(i[1][0])+","
                        else:                       
                            _="["
                            for k in i[1]:
                                if len(k)!=extdegree:k=k+[0]*(extdegree-len(k))
                                _=_+chars[extdegree>1][0]+"".join(hex(a)+",\n\t " for a in k)[:-4]+chars[extdegree>1][1]+",\n\t"
                            _=_[:-3]+"\n\t],"
                res=res+("\n" if res!="{" else "")+'"'+str(i[0])+'":'+_
    return res[:-1]+"}"

def findIso(E):
    for p_test in primes(100):
        isos = [ i for i in isd.isogenies_prime_degree(E, p_test)
            if i.codomain().j_invariant() not in (0, 1728) ]
        if len(isos) > 0:
            return isos[0].dual()
    return None
   
def findZforSSWU(F, A, B):
    R.<xx> = F[]                                                # Polynomial ring over F
    g = xx^3 + F(A) * xx + F(B)                                 # y^2 = g(x) = x^3 + A * x + B
    ctr = F.gen()
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            if is_square(Z_cand):continue                       # Criterion 1: Z is non-square in F.
            if Z_cand == F(-1):continue                         # Criterion 2: Z != -1 in F.            
            if not (g - Z_cand).is_irreducible():continue       # Criterion 3: g(x) - Z is irreducible over F.            
            if is_square(g(B / (Z_cand * A))):return Z_cand     # Criterion 4: g(B / (Z * A)) is square in F.
        ctr += 1

def swuData(field,CurveA,CurveB,toTowerHandle=None):
    extendToDegree =lambda x,degree:x+[0]*(field.degree()-len(x)) if len(x)<degree else x
    result={}
    _basefield=field.base_ring().characteristic()
    _degree=field.degree()
    iso=findIso(EllipticCurve(field, [CurveA, CurveB]))        
    result["Degree"]=[iso.degree()]
    _Z=extendToDegree(list(ZZR(findZforSSWU(field,iso.domain().a_invariants()[3],iso.domain().a_invariants()[4]))),_degree)
    result["Z"]=_Z if toTowerHandle is None else toTowerHandle(_Z,_basefield)
    _isoA=extendToDegree(list(ZZR(iso.domain().a_invariants()[3])),_degree)
    _isoB=extendToDegree(list(ZZR(iso.domain().a_invariants()[4])),_degree)
    result["swuA"]=_isoA if toTowerHandle is None else toTowerHandle(_isoA,_basefield)
    result["swuB"]=_isoB if toTowerHandle is None else toTowerHandle(_isoB,_basefield)
    (xm, ym) = iso.rational_maps()

    names= ("Xnum", "Xden", "Ynum", "Yden")
    maps = (xm.numerator(), xm.denominator(), ym.numerator(), ym.denominator())
    for m in maps:                
        idx=maps.index(m)
        lst=list(m.dict().items())
        _res=[]
        for i in lst :
            _x=extendToDegree(list(ZZR(i[1])),_degree)
            _res=_res+[_x if toTowerHandle is None else toTowerHandle(_x,_basefield)]
        result[names[idx]]=_res
    return result

In [2]:
# BLS24-315 Curve
# x0 = -2^32 + 2^30 + 2^22 - 2^20 + 1 =-0xbfcfffff
prime = 0x4C23A02B586D650D3F7498BE97C5EAFDEC1D01AA27A1AE0421EE5DA52BDE5026FE802FF40300001
F4.<v> = GF(prime^4, modulus=x^4-13)
# Transformation from/to towering 2-2 to 1-4 for the BLS24 Twiste Curve field
fromTower = lambda x,p:[x[0] % p,x[2] % p,(x[1]) % p,(x[3]) % p] # Phi
toTower = lambda x,p:[x[0] % p,x[2]% p,x[1] % p,(x[3]) % p] # Ph^-1
# Twist Curve of G2 parametre B on the originel tower
BtwOriginal = [0,0,0,0xbb6b62e0d9aad15bafe3ee23ebbfcc49a7a9dcb688f071453fd497bdf5d476875ec56258a4ec4f]
TransformedBtw = fromTower(BtwOriginal,prime)
parmG2 = swuData(F4,0,TransformedBtw,toTower)
print("Isogeny data for the BLS24-315: G2 curve is y^2=x^3 + BtwOriginal \n")
print(swudataToStr(parmG2,prime,F4.degree()))

Isogeny data for the BLS24-315: G2 curve is y^2=x^3 + BtwOriginal 

{"Degree":5,
"Z":[-8,0,-1,0],
"swuA":[0,0x3847b70a1a9b8ae12d5537800740b100a98ad16ef40d2907dfafac65d7a110da42af51ae8787fdb,0,0],
"swuB":[0,0,0,0x195508a43cf0850a5a6e24bf3b92fc0ff5f1ec14fbae031d6ea1e1339c298e032fd47ae61aaaafe],
"Xnum":[[0x0,
	 0x0,
	 0x2dc82db4e7938838584fbe950ab48d8056c3165ede5c9019b8ed2b5a06819af2453b36787e6126f,
	 0x0],
	[0x14db8384d71c18702c2e4133d54a7c0a80c0db1e9519f0824f704b30ac2ff12c7c5c8411d2d744a,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x3d43f8428861a3d0384da4ff339f35be0ed2366e06577bcfc743f2e28df1045893b8de980f6da65],
	[0x0,
	 0x4b134455e21797c0a92550a2c600877f75e3f76d5f000074b3a03634848583af1b29c5c0ac8fe05,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x394af986255168c37613065b4ce272e9d1d9d76187e6eb59a8ff371e41a14edab72e20005687722,
	 0x0],
	[0x3ff4f72ea6705f1f9bb3dc77235454c0d0933ed69c2ba6a74fb3bf4d4dcf391683dc513d9c1eb86,
	 0x0,
	 0x0,
	 0x0]
	],
"Xden":[[0x3638b473f6e5f17dd22619c19fe4ee0d79ccd5670e2

In [24]:
# BLS24-477 Curve
# x0 = -0xdfff80000080
prime = 0x167278FAC63BD5B007EBB8F693A2AB3DBD9F92CF437C399D928E94BFE9A04A009FDA9E8CF9226901DE62AEA9DCEA48BF1A0EBBF8860A5E7AD000152B
F4.<v>=GF(prime^4, modulus=x^4-2*x^2+2)
fromTower=lambda x,p:[(x[0]-x[1]) % p,(x[2]-x[3])% p,(x[1]) % p,(x[3]) % p]       # Transformation from towering 2-2 to 1-4 for the BLS24 Twiste Curve field
toTower=lambda x,p:[(x[0]+x[2]) % p,(x[2])% p,(x[1]+x[3]) % p,(x[3]) % p]         # Transformation from towering 1-4 to 2-2 for the BLS24 Twiste Curve field
# Twist Curve of G2 parametre B on the originel tower
BtwOriginal = [0,0,4,0]
TransformedBtw = fromTower(BtwOriginal,prime)
parmG2 = swuData(F4,0,TransformedBtw,toTower)
print("Isogeny data for the BLS24-477: G2 curve is y^2=x^3 + BtwOriginal \n")
print(swudataToStr(parmG2,prime,F4.degree()))

Isogeny data for the BLS24-477: G2 curve is y^2=x^3 + BtwOriginal 

{"Degree":2,
"Z":[22,0,1,0],
"swuA":[30,-30,0,0],
"swuB":[0,0,88,0],
"Xnum":[[0xb393c7d631dead803f5dc7b49d1559edecfc967a1be1ccec9474a5ff4d025004fed4f467c913480ef315754ee75245f8d075dfc43052f3d68000a97,
	 0xb393c7d631dead803f5dc7b49d1559edecfc967a1be1ccec9474a5ff4d025004fed4f467c913480ef315754ee75245f8d075dfc43052f3d68000a94,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0xb393c7d631dead803f5dc7b49d1559edecfc967a1be1ccec9474a5ff4d025004fed4f467c913480ef315754ee75245f8d075dfc43052f3d68000a96,
	 0xb393c7d631dead803f5dc7b49d1559edecfc967a1be1ccec9474a5ff4d025004fed4f467c913480ef315754ee75245f8d075dfc43052f3d68000a96],
	[0x59c9e3eb18ef56c01faee3da4e8aacf6f67e4b3d0df0e6764a3a52ffa68128027f6a7a33e489a407798abaa773a922fc683aefe2182979eb400054b,
	 0x0,
	 0x0,
	 0x0]
	],
"Xden":[[0x0,
	 0x0,
	 0x2,
	 0x2],
	[0x1,
	 0x0,
	 0x0,
	 0x0]
	],
"Ynum":[[0x0,
	 0x0,
	 0x167278fac63bd5b007ebb8f693a2ab3dbd9f92cf437c399d928e94bfe9a04a009fda9e8cf9226901de6

In [25]:
# BLS24-509 Curve
# x0 = -0x800000ffff801
prime = 0x155556FFFF39CA9BFCEDF2B4F9C0ECF6CB8AC8495D187E8C32EA0103E01090BB626E85BF7C18A0F0CFCB5C6071BAD3D2EE63BD076E8D9300A13D118DB8BFD2AB
F4.<v>=GF(prime^4, modulus=x^4-2*x^2+2)
fromTower=lambda x,p:[(x[0]-x[1]) % p,(x[2]-x[3])% p,(x[1]) % p,(x[3]) % p]       # Transformation from towering 2-2 to 1-4 for the BLS24 Twiste Curve field
toTower=lambda x,p:[(x[0]+x[2]) % p,(x[2])% p,(x[1]+x[3]) % p,(x[3]) % p]         # Transformation from towering 1-4 to 2-2 for the BLS24 Twiste Curve field
# Twist Curve of G2 parametre B on the originel tower
BtwOriginal = [0,0,0xaaaab7fff9ce54dfe76f95a7ce0767b65c56424ae8c3f4619750081f008485db13742dfbe0c507867e5ae3038dd69e97731de83b746c980509e88c6dc5fe956,
                0xaaaab7fff9ce54dfe76f95a7ce0767b65c56424ae8c3f4619750081f008485db13742dfbe0c507867e5ae3038dd69e97731de83b746c980509e88c6dc5fe955]
TransformedBtw = fromTower(BtwOriginal,prime)
parmG2 = swuData(F4,0,TransformedBtw,toTower)
print("Isogeny data for the BLS24-509: G2 curve is y^2=x^3 + BtwOriginal \n")
print(swudataToStr(parmG2,prime,F4.degree()))

Isogeny data for the BLS24-509: G2 curve is y^2=x^3 + BtwOriginal 

{"Degree":7,
"Z":[0,0,1,0],
"swuA":[0x117f30ebe37742d0b620ce0953a825403bbf1681bc664e52086bf6b2f4b8f8bab48e759986fa3bf404e0a8fc3392122733308e5ec7de8b585bbd6c3119820680,0x94c796233eacd83cd54254b88ecccb07ca1d99403eaa3b39150885b21bad349cf4a75f995b16dc62425d448a1ddf002e293e2697d4fe34a3997950f7d972d43,0,0],
"swuB":[0,0,0x13e3eb835d7d5e9f279977a6be82b4ed2cc12b47b3271fb6c8e30e06e75130127c50d3bd455b52f847d62600acae988d9b72916350df099243658dc7aa032e77,0x13e3eb835d7d5e9f279977a6be82b4ed2cc12b47b3271fb6c8e30e06e75130127c50d3bd455b52f847d62600acae988d9b72916350df099243658dc7aa02e1ae],
"Xnum":[[0x0,
	 0x0,
	 0x73568d4223aacb0a0f4115dc59213bdad0d2c0ed849b47f3b35178d4dad803f263db3d04911d0067ed53f35429d3c26d933b9ec49580421f07ee885258b6929,
	 0xf4d36373f0472ffc1e6c2f4a0d33abe1f5e2d8acb71ddb258e563275816ffac56af9ccea4c42da2a6e2a6ff8400a404d0dccc001c47398e42f236a8e07ef58],
	[0x14277e7ddce6760bd355ec8011ffaba28e841a8d1527c852640af6b64031a4

In [26]:
# BLS24-509-SNARK Curve
# x0 = 0x811e000000001
prime = 0x17452A017CBDD682A502A1E13A9D671D27958EECD2C33C3A36C2ADE221D9B956BEA4F49B2B5EE7D7D72AD20065DCB2B8E9CA0015B5152C00000811E000000001
F4.<v>=GF(prime^4, modulus=x^4+5)
fromTower = lambda x,p:[x[0] % p,x[2] % p,(x[1]) % p,(x[3]) % p] # Phi  Transformation from towering 2-2 to 1-4 for the BLS24 Twiste Curve field
toTower = lambda x,p:[x[0] % p,x[2]% p,x[1] % p,(x[3]) % p] # Ph^-1  Transformation from towering 1-4 to 2-2 for the BLS24 Twiste Curve field
# Twist Curve of G2 parametre B on the originel tower
BtwOriginal = [0,0,0,0xdf64c674ad84d819634c787232b3dde4af355c14b41f0efba74cebae11c3c00d8c95f904d38f1818119b133705138088c46000d063fe73333380ab99999999a]
TransformedBtw = fromTower(BtwOriginal,prime)
parmG2 = swuData(F4,0,TransformedBtw,toTower)
print("Isogeny data for the BLS24-509-SNARK: G2 curve is y^2=x^3 + BtwOriginal \n")
print(swudataToStr(parmG2,prime,F4.degree()))

Isogeny data for the BLS24-509-SNARK: G2 curve is y^2=x^3 + BtwOriginal 

{"Degree":5,
"Z":[-6,0,-1,0],
"swuA":[0x1d8055b0b68de1d9ee33f9e14d472c0f1a4a86703863a119f71a21e2b291a0053ee731801de0728dd423fa0add330e22b52258679635219e6592025e806beb4,0x146010cbaa62eb67b46e733e846405a3fc7e1e89c1ea5986ee232cf97fdfeb4c29b32a1171104694f1e1b37848a8877639b5a597447d08518218630f1a18ce82,0,0],
"swuB":[0,0,0x15db44669674fcfc2eedf68006d81fc2310a8d2aa0e78f6a5391bb3559b79c17522ccb9322bd40502e6f833c2ba40f43d4a65b21f54c6f1faf81af1a32a847c,-521],
"Xnum":[[0x0,
	 0x0,
	 0x7e7ff9b0e5fb0176756652607f2d2ef9dec58973fffced61825d6a3d840bd8044491690033734607e02f98ad88da66162f27abe177916860fa59ea90beeeced,
	 0x167d890391f7db4e00dbb21c9121993f603a4b39fb23c3f6da2a5b83ecee687452a2b35e87bd2b5281a3212a52f83759253dde40093b4fc0c91eb946b024df50],
	[0xee5196273515d2333b59bbbe584fa3320162a5c73fddc9ae7304983fd149580281d2f7ef3255be7f95e40669ab9afe8f203b0756b2c6e6bc5a1dbc696b1a0df,
	 0xfbbacfb8fc19c0fe46b620d920f559e1aa77fd85a34056

In [27]:
# BLS24-559 Curve
# x0 = -0x10007FFFFFFFE40
prime = 0x557003C04FFE89DB51EF86C56921091E1C72D1E92397D41E37C255CD44EFF051C38FA813A695C5D574923BEEE353ADA1100A0B5FB2111A277D389362D77FCB581FE8B50105EB
F4.<v>=GF(prime^4, modulus=x^4-2*x^2+2)
fromTower=lambda x,p:[(x[0]-x[1]) % p,(x[2]-x[3])% p,(x[1]) % p,(x[3]) % p]       # Transformation from towering 2-2 to 1-4 for the BLS24 Twiste Curve field
toTower=lambda x,p:[(x[0]+x[2]) % p,(x[2])% p,(x[1]+x[3]) % p,(x[3]) % p]         # Transformation from towering 1-4 to 2-2 for the BLS24 Twiste Curve field
# Twist Curve of G2 parametre B on the originel tower
BtwOriginal = [0,0,0x557003C04FFE89DB51EF86C56921091E1C72D1E92397D41E37C255CD44EFF051C38FA813A695C5D574923BEEE353ADA1100A0B5FB2111A277D389362D77FCB581FE8B50105EA,
                0X1]
TransformedBtw = fromTower(BtwOriginal,prime)
parmG2 = swuData(F4,0,TransformedBtw,toTower)
print("Isogeny data for the BLS24-559: G2 curve is y^2=x^3 + BtwOriginal \n")
print(swudataToStr(parmG2,prime,F4.degree()))

Isogeny data for the BLS24-559: G2 curve is y^2=x^3 + BtwOriginal 

{"Degree":2,
"Z":[0,0,1,0],
"swuA":[0xf04388709d789d5a5dec3fd9c4da418dff8fe4235ffa11ffcbf63cf503cc5a3e04fd9b83ad2c41b45fe352263a405ed0ca22f7432453925abc57529e06c3a22,0xf04388709d789d5a5dec3fd9c4da418dff8fe4235ffa11ffcbf63cf503cc5a3e04fd9b83ad2c41b45fe352263a405ed0ca22f7432453925abc57529e06c3a22,0,0],
"swuB":[0,0,-22,22],
"Xnum":[[0x2ab801e027ff45addefe83e0aea8000d64d2e3f87399f541f5647a669db99198d9f8980cdd2cae21eb92541b95119a5071138aab3fbacdb5742ee5ce7b682e42bc367285ec44,
	 0x2ab801e027ff45addefe83e0aea8000d64d2e3f87399f541f5647a669db99198d9f8980cdd2cae21eb92541b95119a5071138aab3fbacdb5742ee5ce7b682e42bc367285ec44,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x2ab801e027ff446d84f3430eb880dce57f286c475097e297e0344b3bfaf6e73391a7515d2209b0c5eec2f9df5a0f547b41fb57b2e9e70ca7f03a8c48b6a50a9cf272f5279216],
	[0x155c00f013ffa276d47be1b15a484247871cb47a48e5f5078df09573513bfc1470e3ea04e9a571755d248efbb8d4eb68440282d7ec844689df4e24d8b5d

In [28]:
# BLS48-277 Curve
# x0 = -0xadc0
prime=0x14674E388DA524ED849C38E2B187AF9D0AB5A06A924839098277FD01150000274ECB6B  
F8.<w>=GF(prime^8,modulus=x^8+2*x^4+2)
fromTowerF8=lambda x,p:[(x[0]-x[1]) % p,(x[4]-x[5])% p,(x[3]-x[2])% p,(x[7]-x[6])% p,-(x[1]) % p,-(x[5]) % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[(x[0]+x[4]) % p,-(x[4])% p,(x[6]-x[2]) % p,(x[6]) % p,(x[1]-x[5]) % p,-(x[5]) % p,(x[7]-x[3]) % p,(x[7]) % p]  
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,4,0,0,0]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-277: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-277: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":3,
"Z":[7,0,0,0,1,0,0,0],
"swuA":[0,0,120,120,0,0,0,0],
"swuB":[0,0,0,0,1012,0,0,0],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x488bc0c914124a68f3f2915ee8fee22e5ef78fb3cf39ae5ab371bc75a000008bc2d56,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x6cd1a12d9e1b6f9d6debda0e5d7e53458e73578db6d685880d2a9ab0700000d1a43cd,
	 0x6cd1a12d9e1b6f9d6debda0e5d7e53458e73578db6d685880d2a9ab0700000d1a43cd,
	 0x0,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0xd9a3425b3c36df3adbd7b41cbafca68b1ce6af1b6dad0b101a553560e00001a34879e],
	[0x1222f0324504929a3cfca457ba3fb88b97bde3ecf3ce6b96acdc6f1d68000022f0b4d1,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0]
	],
"Xden":[[0x0,
	 0x0,
	 0x14674e388da524ed849c38e2b187af9d0ab5a06a924839098277fd01150000274ecb47,
	 0x14674e388da524ed849c38e2b187af9d0ab5a06a924839098277fd01150000274ecb47,
	 0x0,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0xc],
	

In [29]:
# BLS48-287 Curve
# x0 = -0xfe5a
prime=0x4BF7DE6E1D51DE83959E838FBF22B9C69A0BDDB43A92FFE58991E030D59F59012948C019        
F8.<w>=GF(prime^8,modulus=x^8-7)
fromTowerF8=lambda x,p:[x[0] % p,x[4] % p,x[2] % p,x[6] % p,x[1] % p,x[5] % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[x[0] % p,x[4] % p,x[2] % p,x[6] % p,x[1] % p,x[5] % p,(x[3]) % p,(x[7]) % p]       
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,0,0,0,0xada4458df9dfb375e845beff6bbd16583b88d62e3cbdb69ef14d6e267a90cb705e5d24d]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-287: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-287: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":13,
"Z":[13,0,0,0,1,0,0,0],
"swuA":[0x474caefa5b612929c9e239aba5d72f19f739532d4cf4ad1e020a118c43bfd2765ab6eeda,0x31a78c06d8223a6fc00bd78c5e6f74cad8224c7bb54cba2a1eb0a7a404da24d7956490c5,0x1048cde3280ee084fa26447295733dd1333acb68e764f351a407c4b585204786791b2320,0x4284f3fdd5b94174590d2223abc69079f607b778902a1a1575a270300c6d671c9a91ee26,0,0,0,0],
"swuB":[0,0,0,0,0x18d2f0587736db2a7eef4b7acdc85fe95801affffffd382f868f94fa48f932b0c777c760,0x3e323aa8606044bfc814820aa9a823c4e960e869b21576f8e4ed75dd95ab2988e2a83673,0x1148daa978e93f82562e4002baaa76b4d42293f978f6014be70e1291cf98f30e9f7405bb,0xada4458df9dfb375e845beff6bbd16583b88d62e3cbdb69ef14d6e267a90cb705e7932d],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0xd9db13db3c97dceeb9289ca62afe5f044880d77064457bbb416dff5749f821477c2c0bc,
	 0x4b2001356ec237ec4c24809ea1045acc2e4fc707c74f39d86ad0345fe65a8f85264cba6b,
	 0x47edec52963e29dcc9bf3870d6f16f62c67e8ccfee9cd6b0100d93238b367d0d2a7

In [30]:
# BLS48-571 Curve
# x0 = -0xDFF82003
prime=0x7B1D5F53AB93DA0DFB9F21560B2B742699F19B31171DC1F4F2FE9A7C17C8DFCDA3BBDF9472A6FF9146F545249F4B8573A2E89B92214F7C4E00910FBFCBF22A81C0F860053BC28AD        
F8.<w>=GF(prime^8,modulus=x^8-2)
fromTowerF8=lambda x,p:[x[0] % p,x[4] % p,x[2] % p,x[6] % p,x[1] % p,x[5] % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[x[0] % p,x[4] % p,x[2] % p,x[6] % p,x[1] % p,x[5] % p,(x[3]) % p,(x[7]) % p]       
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,0,0,0,
             0x3D8EAFA9D5C9ED06FDCF90AB0595BA134CF8CD988B8EE0FA797F4D3E0BE46FE6D1DDEFCA39537FC8A37AA2924FA5C2B9D1744DC910A7BE27004887DFE5F91540E07C30029DE1457]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-571: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-571: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":3,
"Z":[3,0,0,0,1,0,0,0],
"swuA":[0,0,0x7b1d5ef0b9a39ad40ba276bbfe2328ce859315a2df3db4e6fc29c16f6713513eaa0df47efbc442a4bca0da409637b0cf355f3a278ffc561789b4b90c3a24b98f384038c3ead1f35,0,0,0,0,0],
"swuB":[0,0,0,0,0,0,0,0x3d8eafa9d5c9ed06fdcf90ab0595ba134cf8cd988b8ee0fa797f4d3e0be46fe6d1ddefca39537fc8a37aa2924fa5c2b9d1744dc910a7be27004887dfe5f91540e07c30029de14d5],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x5fc18308301da9991909fd7bcfcc76c8b09f78b4675008a212543f44127fca837f593c1e2048ffa9e1db35c726901276624323aa8baf99920070d35c494a92d6b2884aaebcb3c19],
	[0x0,
	 0x0,
	 0x2909ca7252a2e2fcc57945e581060b1c116857590e03e01612518b3d6a9a4f078eaf7d7e8567b24f7f96f61a1e719266f340d7129af0c09d73498ebf153f17f50ea47b7717a9ebb,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x521394e38bd42bac1958fbacda148728ef638b1415b87612635114117287eef6c54372b000f4b22a973e0d26538abee2d4390aedfb6094b773

In [31]:
# BLS48-573 Curve
# x0 = -0xebb80000
prime=0x1350A0C8F4218394626944EAAACAD46B7D9DCCAF31B20F740ED2116B9AE102823CC9418DC9618006091711D2F3FAEAE45DCF869195000000000000000000000048591C1506C2AAAB        
F8.<w>=GF(prime^8,modulus=x^8-2*x^4+2)
fromTowerF8=lambda x,p:[(x[0]-x[1]) % p,(x[4]-x[5])% p,(x[2]-x[3])% p,(x[6]-x[7])% p,(x[1]) % p,(x[5]) % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[(x[0]+x[4]) % p,(x[4])% p,(x[2]+x[6]) % p,(x[6]) % p,(x[1]+x[5]) % p,(x[5]) % p,(x[3]+x[7]) % p,(x[7]) % p]     
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,4,0,0,0]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-573: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-573: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":3,
"Z":[-8,0,0,0,-1,0,0,0],
"swuA":[0,0,120,120,0,0,0,0],
"swuB":[0,0,0,0,1012,0,0,0],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x44ace658b95ab764ec20f5097bb4ba61bea2d7c43eeae19ca679217e98755e40d820e9149321c731e76e7843637c26ba311ac205a00000000000000000000001013cd5a018097d6,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x670359851608131762316f8e398f17929df443a65e60526af9b5b23de4b00d6144315d9edcb2aacadb25b465153a3a1749a8230870000000000000000000000181db4070240e38d,
	 0x670359851608131762316f8e398f17929df443a65e60526af9b5b23de4b00d6144315d9edcb2aacadb25b465153a3a1749a8230870000000000000000000000181db4070240e38d,
	 0x0,
	 0x0,
	 0x0,
	 0x0],
	[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x670359851608131762316f8e398f17929df443a65e60526af9b5b23de4b00d6144315d9edcb2aacadb25b465153a3a1749a8230870000000000000000000000181db4070240e38d],
	[0x112b39962e56add93b083d425eed2e986fa8b5f10fbab867299e485fa61d579036083a4524c871cc79db9e1

In [32]:
# BLS48-575 Curve
# x0 = -0xff800801
prime=0x526222098BE0D6809EC22A43AB79D1F2120ECBCFBAB8934E6C19B1B30A442EEA74C92F3ED3940030DBE05F531CD41C69464C793821806366EC094551C2828E22D811DCC31BEB02AB        
F8.<w>=GF(prime^8,modulus=x^8-2*x^4+2)
fromTowerF8=lambda x,p:[(x[0]-x[1]) % p,(x[4]-x[5])% p,(x[2]-x[3])% p,(x[6]-x[7])% p,(x[1]) % p,(x[5]) % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[(x[0]+x[4]) % p,(x[4])% p,(x[2]+x[6]) % p,(x[6]) % p,(x[1]+x[5]) % p,(x[5]) % p,(x[3]+x[7]) % p,(x[7]) % p]     
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,1,0,0,0]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-575: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-575: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":5,
"Z":[0,0,0,0,-1,0,0,0],
"swuA":[0,0,0x4dda7df5efdc40f6aefacabff6f13750226f3dc532ba00acb333447b1016e90a2ddc9c8ea0b8d2a4af34fe7747289fadf296399c5a4252c34c71ea9fe098c0c49d946cf179a76321,0x4dda7df5efdc40f6aefacabff6f13750226f3dc532ba00acb333447b1016e90a2ddc9c8ea0b8d2a4af34fe7747289fadf296399c5a4252c34c71ea9fe098c0c49d946cf179a76321,0,0,0,0],
"swuB":[0,0,0,0,0x388e62090bfed90180182594813dca7b6c2ee485aaa6bb4a7a852c2e75c0a57e14cdf703dfecd975c0282cb9d6ec954badb4728575c905eb301c5b164590da6561c5ceec8f337a5f,0,0,0],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x3c75a7d12ddbff8b316e941c849f5975c992922d58673b4113491b49d4587f68c0f74a9958b0b84199cff81190ea73916107bb583f6e92fdc0a70109cf8e05606db76e88390b0def,
	 0x3c75a7d12ddbff8b316e941c849f5975c992922d58673b4113491b49d4587f68c0f74a9958b0b84199cff81190ea73916107bb583f6e92fdc0a70109cf8e05606db76e88390b0def],
	[0x4c134275874673c778778c0996c88deb85eccc7215eebee17fa9515c

In [33]:
# BLS48-581 Curve
# x0 = -0x140000381
prime = 0x1280F73FF3476F313824E31D47012A0056E84F8D122131BB3BE6C0F1F3975444A48AE43AF6E082ACD9CD30394F4736DAF68367A5513170EE0A578FDF721A4A48AC3EDC154E6565912B  
F8.<w>=GF(prime^8,modulus=x^8+2*x^4+2)
fromTowerF8=lambda x,p:[(x[0]-x[1]) % p,(x[4]-x[5])% p,(x[3]-x[2])% p,(x[7]-x[6])% p,-(x[1]) % p,-(x[5]) % p,(x[3]) % p,(x[7]) % p]       # Transformation from towering 2-2-2 to 1-8 for the BLS48 Twiste Curve field
toTowerF8=lambda x,p:[(x[0]+x[4]) % p,-(x[4])% p,(x[6]-x[2]) % p,(x[6]) % p,(x[1]-x[5]) % p,-(x[5]) % p,(x[7]-x[3]) % p,(x[7]) % p]  
# Twist Curve of G2 parametre B on the originel tower F8<w>=ExtensionField<Fp4,v|w^2-v> of 
BtwOriginal=[0,0,0,0,0,0,0x9407b9ff9a3b7989c12718ea38095002b7427c6891098dd9df36078f9cbaa225245721d7b7041566ce6981ca7a39b6d7b41b3d2a898b877052bc7efb90d2524561f6e0aa732b2c895,
             0x9407b9ff9a3b7989c12718ea38095002b7427c6891098dd9df36078f9cbaa225245721d7b7041566ce6981ca7a39b6d7b41b3d2a898b877052bc7efb90d2524561f6e0aa732b2c896]
TransformedBtw=fromTowerF8(BtwOriginal,prime)
parmG2=swuData(F8,0,TransformedBtw,toTowerF8)                 # toTower handel transformation back to the 2-2-2 towering scheme
print("Isogeny data for the BLS48-581: G2 curve is y^2=x^3+BtwOriginal \n")
print(swudataToStr(parmG2,prime,F8.degree()))

Isogeny data for the BLS48-581: G2 curve is y^2=x^3+BtwOriginal 

{"Degree":3,
"Z":[6,0,0,0,1,0,0,0],
"swuA":[0,0,0,0xa688af6d41fe1d3564fa335328e12ac3cae641571e73cba63d48f7092a6b0ff303822a6f2b13b940adb86ab369e3201bf5f69f37e1f95c2d56e415e0268a42c4660eac875c0,0,0,0,0],
"swuB":[0,0,0,0,0,0,0x9407b9ff9a3b7989c12718ea38095002b7427c6891098dd9df36078f9cbaa225245721d7b7041566ce6981ca7a39b6d7b41b3d2a898b877052bc7efb90d2524561f6e0aa732b2c817,0x9407b9ff9a3b7989c12718ea38095002b7427c6891098dd9df36078f9cbaa225245721d7b7041566ce6981ca7a39b6d7b41b3d2a898b877052bc7efb90d2524561f6e0aa732b2c914],
"Xnum":[[0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x0,
	 0x41ca8b8e0ba8a7cb724a43f651cb400135011ad924043f0b7fa63c3fd3e12b9ebad1645fdf9009830667c7e836528a26fab6c5da043e034e5da91bfe0779ebad0f1863d9fa4c13cb0,
	 0xe644e87128ce4b481003edde1e4760043983ddf7fe0edca83ec5d2df659418ab8ddcdf4f8e78214a966b3bacbe20e3886d7fb47b0ed90b9247cfe1f91a2ab8ddb4d55d7aec0a4547b],
	[0x0,
	 0x0,
	 0x0,
	 0xc55fa2a84947a95ec2faea95095b7daafd38e